## 偏见发现

### 想法来源

在持有海信家电的过程中，直觉上感到，市场每次对海信家电的财报预期都不好，但是出财报之后常常能超预期并迎来上涨，也就是市场可能对它<b>存在偏见</b>。最近又出现了这种情况。

但是直觉不靠谱，需要用数字检验。

### 检验定义
对给定股票，统计其在过往8次发财报，前后15天、30天内的涨跌幅，观察是否有规律。

8次一般对应2年时间，毕竟如果真有这种规律，也不可能持续太长时间。15天和30天的选择也是基于类似考虑，毕竟一般2次财报之间的间隔才3个月。

In [1]:
import pandas as pd


def get_price_d(security, date):
    price = pd.DataFrame()
    while price.shape[0] == 0:
        # print(date)
        ## 这里有个问题，获取的收盘价跟东财不一致，7月的是一致的，但5月就不一致了
        price = get_price(security, start_date=date, end_date=date, frequency='daily', fields=['close'], skip_paused=False, fq='pre', panel=False)
        date = date + datetime.timedelta(-1)
    return(price.iloc[0,0])

def cal_ratio(price_1, price_2):
    return (price_2 - price_1) / price_1


def observe_bias(stock, date_report, interval):
    
    ratio_matrix = []
    for date in date_report:
        # 未找到日历函数，选择麻烦的方法
        prices = []
        
        date_1 = datetime.datetime.strptime(date, '%Y-%m-%d')
        date_start = date_1 + datetime.timedelta(-interval)
        date_end = date_1 + datetime.timedelta(interval)
        for dd in [date_start, date_1, date_end]:
            price = get_price_d(stock, dd)
            # print(price)
            prices.append(price)
        
        ratio = [cal_ratio(prices[0], prices[1]), cal_ratio(prices[1], prices[2])] 
        ratio_matrix.append(ratio)
        
    return ratio_matrix

In [73]:
def get_report_time(security):
    if not (security[0:2] in ['00','30','60','68']):
        return None
    # 周期两年
#     end_date = datetime.datetime.now()
#     start_date = end_date + datetime.timedelta(-366 * 2)
#     print(start_date, end_date)
    
    date_list = []
    years = ['2019','2020','2021']
    for year in years:
        for ii in np.arange(1,5,1):
            date_list.append(year+'q'+str(ii))
    del date_list[-3:]
    del date_list[0]
    
    time_list = []
    for date in date_list:
        q = query(income.code, income.pubDate).filter(
            income.code == security).limit(3000)
        df = get_fundamentals(q, statDate=date)
        df = df.iloc[0,1]
        # df = df['pubDate'].drop_duplicates().values.tolist()
        time_list.append(df)
        
    return time_list                                       

In [58]:
stock = '000921.XSHE'
index = '399001.XSHE'

# 对其它股票，用函数get_report_time()获取时间列表
date = ['2019-8-23','2019-10-29','2020-4-15','2020-4-29','2020-8-21','2020-10-29','2021-3-31','2021-4-29']

# 记得修改时间间隔
rate_secu = observe_bias(stock, date, 15)
rate_index = observe_bias(index, date, 15)

In [3]:
# 平均超额收益
car = 0
for ii in range(len(rate_secu)):
    car += (rate_secu[ii][1] - rate_index[ii][1])
car /= (len(rate_secu))
print("平均超额收益：   ", car/(len(rate_secu) ))
# '000001.XSHG'

平均超额收益：    -0.0033955201482623243


In [4]:
# pearson
rate_secu_n = np.array(rate_secu)
rate_index_n = np.array(rate_index)

corr_self = np.corrcoef(rate_secu_n[:,0], rate_secu_n[:,1])
corr_index_after = np.corrcoef(rate_secu_n[:,1], rate_index_n[:,1])
corr_index_before = np.corrcoef(rate_secu_n[:,0], rate_index_n[:,0])

print(corr_self[0,1], corr_index_after[0,1], corr_index_before[0,1])

-0.11235803506574697 0.4073889096583994 -0.3171160448284253


### 分析
1. 平均超额收益
    - 后15天平均超额收益：    -0.0033955201482623243
    - 后30天平均超额收益：    -0.002588250391463465
2. 与大盘走势相关性/假设检验，例如pearson相关系数
    - 后15天股票走势与前15天相关系数-0.11，与大盘走势相关0.4，算是有一点点特征，但也毫无用处
    - 后30天股票走势与前30天的相关性，后30天股票走势与大盘的相关性，都大约是0.3，不能说明任何问题
3. 绘制并列直方图
    - 算了，懒得画了
    
### 总结
所谓直觉，都是一厢情愿，数字统计表明，没什么关系

未来也许可以查看其它股票是否长期存在财报后稳定上涨的情况，也能检验机构们对这只股票的研究深度